### Cluster human walking motions

In this experiment we cluster human and running walking human frames into two classes.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
import seaborn as sns
import torch
import torch.distributions as dist
from signatureshape.animation import fetch_animation_id_set, fetch_animations
import extratorch as etorch
import shapeflow as sf

In [ ]:
# make reproducible
seed = torch.manual_seed(0)

# better plotting
set_matplotlib_formats("pdf", "svg")
matplotlib.rcParams.update({"font.size": 12})
set_matplotlib_formats("pdf", "svg")
plt.style.use("tableau-colorblind10")
sns.set_style("white")

Load and standardize motion capture data

In [ ]:
# load data as euler angles
# we assume all have the same skeleton
print("Loading mocap data:")
# walk  data
walk_subjects = ["07", "08", "35", "16"]
walk_animations = []
for s in walk_subjects:
    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:4] == "walk":
            walk_animations.append(t[1])

# run data
run_subjects = ["09", "16", "35"]
run_animations = []
run_skeletons = []
for s in run_subjects:
    for t in fetch_animations(100, subject_file_name=(s + ".asf")):
        if t[2][:3] == "run":
            run_skeletons.append(t[0])
            run_animations.append(t[1])

print("Convert to array:")
walk_angle_array = sf.utils.animation_to_eulers(
    walk_animations,
    reduce_shape=False,
    remove_root=True,
    deg2rad=True,
    max_frame_count=240,
)
run_angle_array = sf.utils.animation_to_eulers(
    run_animations,
    reduce_shape=False,
    remove_root=True,
    deg2rad=True,
    max_frame_count=240,
)

In [ ]:
# tensors on the form
# (motion, time, joint)
walk_angle_tensor = torch.tensor(walk_angle_array, dtype=torch.float32)
run_angle_tensor = torch.tensor(run_angle_array, dtype=torch.float32)

pre_shape_walk = walk_angle_tensor.shape
pre_shape_run = run_angle_tensor.shape
num_frames = min(pre_shape_walk[1], pre_shape_run[1])

nonzero = torch.argwhere(
    torch.sum(torch.abs(torch.diff(walk_angle_tensor, dim=1)), dim=[0, 1]) > 0.0
).flatten()

In [ ]:
skip_frames = 12

# insert chosen dimensions here
choosen = nonzero[[11, 23, 30]]

# cut and reduce frames
walk_angles = walk_angle_tensor[:, :num_frames:skip_frames, choosen]
run_angles = run_angle_tensor[:, :num_frames:skip_frames, choosen]
wr_angles = torch.cat((walk_angles, run_angles))

animation_shape = wr_angles.shape[-2:]
animation_shape

In [ ]:
# standardize
std, mean = torch.std_mean(wr_angles, dim=0)
wr_angles_norm = (wr_angles - mean) / std
run_angles_norm = (run_angles - mean) / std
walk_angles_norm = (walk_angles - mean) / std
std.shape

In [ ]:
# reshape data
flatten = True
add_channel = False
make_frames = False
orig_shape_walk = walk_angles.shape
orig_shape_run = run_angles.shape
orig_shape_wr = wr_angles.shape

if add_channel:
    walk_angles_nr = torch.unsqueeze(walk_angles_norm, 1)
    run_angles_nr = torch.unsqueeze(walk_angles_norm, 1)
    wr_angles_nr = torch.unsqueeze(wr_angles_norm, 1)
elif flatten:
    walk_angles_nr = walk_angles_norm.reshape(
        orig_shape_walk[0], orig_shape_walk[1] * orig_shape_walk[2]
    )
    run_angles_nr = run_angles_norm.reshape(
        orig_shape_run[0], orig_shape_run[1] * orig_shape_run[2]
    )
    wr_angles_nr = wr_angles_norm.reshape(
        orig_shape_wr[0], orig_shape_wr[1] * orig_shape_wr[2]
    )
elif make_frames:
    walk_angles_nr = walk_angles_norm.reshape(
        orig_shape_walk[0] * orig_shape_walk[1], orig_shape_walk[2]
    )
    run_angles_nr = run_angles_norm.reshape(
        orig_shape_run[0] * orig_shape_run[1], orig_shape_run[2]
    )
    wr_angles_nr = wr_angles_norm.reshape(
        orig_shape_wr[0] * orig_shape_wr[1], orig_shape_wr[2]
    )
wr_angles_nr.shape

#### Make priors
Since (pretend) we do not know the class of each observation the estimated prior probability is (0.5 + $\epsilon$,0.5 - $\epsilon$) for all observations.


In [ ]:
walk_priors = torch.cat(
    (torch.ones(len(walk_angles_nr)), torch.zeros(len(run_angles_nr)))
)
run_priors = abs(walk_priors - 1)
q = torch.stack((walk_priors, run_priors), dim=1)

#  priors with equal probability
priors = torch.zeros_like(q)
eps = torch.rand(len(priors)) * 0.1
priors[:] = 0.5
priors[:, 1] += eps
priors[:, 0] -= eps

In [ ]:
# make datasets for training
data = torch.utils.data.TensorDataset(wr_angles_nr, priors.clone().detach(), priors)
data_walk = torch.utils.data.TensorDataset(walk_angles_nr)
data_run = torch.utils.data.TensorDataset(run_angles_nr)

Choose model parameters

In [ ]:
#######
DIR = "../figures/cluster_shape/"
SET_NAME = "dim_selection_6_auto"
PATH_FIGURES = os.path.join(DIR, SET_NAME)
########

event_shape = data[0][0].shape
base_dist = dist.Independent(
    dist.Normal(loc=torch.zeros(event_shape), scale=torch.ones(event_shape)), 1
)
lr_scheduler = lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, mode="min", factor=0.5, patience=10, verbose=True
)

joint_shape = (
    animation_shape[-1] if animation_shape[-1] % 2 == 1 else animation_shape[-1] - 1
)
MODEL_PARAMS = {
    "model": sf.nf.get_flow,
    "get_transform": sf.transforms.NDETransform,
    "base_dist": base_dist,
    "get_net": sf.models.CNN2D,
    "activation": "tanh",
    "inverse_model": True,
    "num_flows": 2,
    "sensitivity": ["adjoint"],
    "trace_estimator": "autograd",
    "kernel_size": (3, joint_shape),
    "internal_shape": animation_shape,
    "n_hidden_layers": [4],
}

TRAINING_PARAMS = {
    "batch_size": [20],
    "compute_loss": [sf.nf.get_monte_carlo_conditional_dkl_loss()],
    "verbose": True,
    "optimizer": ["ADAM"],
    "num_epochs": [50],
    "learning_rate": [0.01],
    "lr_scheduler": [lr_scheduler],
}

Do the actual training with model parameters

In [ ]:
# create iterators
model_params_iter = etorch.create_subdictionary_iterator(MODEL_PARAMS, product=True)
training_params_iter = etorch.create_subdictionary_iterator(
    TRAINING_PARAMS, product=True
)

cv_results = etorch.k_fold_cv_grid(
    fit=etorch.fit_module,
    model_params=model_params_iter,
    training_params=training_params_iter,
    data=data,
)

Store and plot results

In [ ]:
etorch.plotting.plot_result(
    path_figures=PATH_FIGURES,
    **cv_results,
)
models = cv_results["models"][0]

Print classification of each class

In [ ]:
motion_data = data_walk[:][0]
print("Walk data:")
print(
    "Class 1:",
    torch.sum(models[0].log_prob(motion_data) < models[1].log_prob(motion_data)).item(),
    "Class 2:",
    torch.sum(models[0].log_prob(motion_data) > models[1].log_prob(motion_data)).item(),
)

motion_data = data_run[:][0]
print("Run data:")
print(
    "Class 1 :",
    torch.sum(models[0].log_prob(motion_data) < models[1].log_prob(motion_data)).item(),
    "Class 2:",
    torch.sum(models[0].log_prob(motion_data) > models[1].log_prob(motion_data)).item(),
)